In [1]:
from src.features.extract_features import *
from src.data.dataset import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = r"F:\01UOttawa\Graduation project\models notebook\Dataset\Semi-final data"

In [3]:
from sklearn.metrics import accuracy_score, recall_score, balanced_accuracy_score, f1_score

def eval_clf(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average = 'macro')
    fscore = f1_score(y_true, y_pred, average = 'macro')
    return acc, balanced_acc, recall, fscore

In [4]:
from lightgbm import LGBMClassifier as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss

In [7]:
model = lgb(learning_rate = 0.05, n_estimators = 2000)#boosting_type = 'rf', learning_rate = 0.05, n_estimators = 2000, bagging_freq = 20, bagging_fraction = 0.5)

# feature_sets = [#['RMS'],
#                 #['ssc', 'zc', 'WL'],
#                 ['mav', 'ssc', 'zc', 'WL']] # Hudgins
# #                 ['RMS', 'ssc', 'zc', 'WL'],
# #                 ['Var', 'ssc', 'zc', 'WL'],
# #                 ['mav', 'RMS', 'ssc', 'zc', 'WL'],
# #                 ['mav', 'Var', 'ssc', 'zc', 'WL'],
# #                 ['mav', 'RMS', 'Var', 'ssc', 'zc', 'WL'],
# #                 ['iEMG', 'WAMP', 'Var', 'ssc', 'zc', 'WL']]
#                 #['skewness', 'RMS', 'iEMG', 'hjorth_params', ''],
#                 #['mmnf', 'mmnf', 'mdf', 'mmdf']]

feature_sets = [['skewness', 'RMS', 'iEMG', 'hjorth_params', 'ar_coefficients', 'mav', 'ssc', 'zc', 'WL']]

In [8]:
for fs in feature_sets:
    dataset = EmgDataset(data_dir, win_size = 200, win_stride = 40, feature_set = fs)
    emg_features = dataset.extracted_features
    labels = dataset.rolled_labels
    reps = dataset.rolled_repetition
    train_rows = np.isin(reps, ['1','3','4']).ravel()
    test_rows = np.isin(reps, ['2']).ravel()

    train_emg = emg_features[train_rows]
    y_train = labels[train_rows].ravel()

    test_emg = emg_features[test_rows]
    y_test = labels[test_rows].ravel()
    
    #under sampling
#     rus = RandomUnderSampler(random_state = 0, sampling_strategy = 'majority')
#     x_res_rus, y_res_rus = rus.fit_resample(train_emg, y_train)
    
#     undersample = NearMiss(version=1, n_neighbors=3)
#     x_res_rus, y_res_rus = undersample.fit_resample(train_emg, y_train)
    
    
    model.fit(train_emg, y_train)
    train_preds = model.predict(train_emg)
    test_preds = model.predict(test_emg)
    train_acc, bal_train_acc, train_recall, train_fscore = eval_clf(y_train, train_preds)
    test_acc, bal_test_acc, test_recall, test_fscore = eval_clf(y_test, test_preds)
    
    print(f'Features: {fs}')
    print(f"train acc: {train_acc} test acc: {test_acc}")
    print(f"balanced train acc: {bal_train_acc} balanced test acc: {bal_test_acc}")
    print(f"train recall: {train_recall} test recall: {test_recall}")
    print(f"train f1_score: {train_fscore} test f1_score: {test_fscore}")
    print('==================================================================================')
    

Feature hjorth_params not supported yet
Feature ar_coefficients not supported yet
Features: ['skewness', 'RMS', 'iEMG', 'hjorth_params', 'ar_coefficients', 'mav', 'ssc', 'zc', 'WL']
train acc: 1.0 test acc: 0.8131604226705091
balanced train acc: 1.0 balanced test acc: 0.7290667969351574
train recall: 1.0 test recall: 0.7290667969351574
train f1_score: 1.0 test f1_score: 0.7305400756819079


In [ ]:
import time
import timeit

def prediction_time(data_features, model):
    pred_t_start = timeit.default_timer()
    pred = model.predict(data_features[:1]) # prediction time for only one record
    pred_t_end = timeit.default_timer()
    print(f'prediction time: {pred_t_end - pred_t_start}')
    

In [ ]:
from collections import Counter

print('y_train')
print(Counter(y_train).keys())
print(Counter(y_train).values())
print('==================================================================================')

print('y under sampling')
print(Counter(y_res_rus).keys())
print(Counter(y_res_rus).values())
print('==================================================================================')

print('y_test')
print(Counter(y_test).keys())
print(Counter(y_test).values())
print('==================================================================================')

print('test_preds')
print(Counter(test_preds).keys())
print(Counter(test_preds).values())
print('==================================================================================')
